# Exploring Hacker News Posts
#### In this project, I'll work with a data set of submissions to the popular technology site [Hacker News](https://news.ycombinator.com/).

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

The data set can be found [here](https://www.kaggle.com/hacker-news/hacker-news-posts), but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Below are descriptions of the columns: 
- `id`: The unique identifier from Hacker News for the post
- `title`: The title of the post
- `url`: The URL that the posts links to, if it the post has a URL
- `num_points`: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
- `num_comments`: The number of comments that were made on the post
- `author`: The username of the person who submitted the post
- `created_at`: The date and time at which the post was submitted


In this project, I'll compare two different types of posts from Hacker News, a popular site where technology related stories (or 'posts') are voted and commented upon. The two types of posts I'll explore begin with either Ask HN or Show HN.

Users submit Ask HN posts to ask the Hacker News community a specific question, such as "What is the best online course you've ever taken?" Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting.

I'll specifically compare these two types of posts to determine the following:

- Do Ask HN or Show HN receive more comments on average?
- Do posts created at a certain time receive more comments on average?


## Introduction
First the data will be read then headers removed

In [11]:
from csv import reader
import datetime as dt

In [3]:
with open('hacker_news.csv', 'r') as file:
    hn = list(reader(file))

hn[:5] #Print top 5 rows

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

## Removing Headers from a List of Lists


In [4]:
headers = hn[0]
print(headers)
#Show first five rows of dataset
hn = hn[1:]
hn[:5]

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01'],
 ['10301696',
  'Note by Note: The Making of Steinway L1037 (2007)',
  'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',
  '8',
  '2',
  'walterbell',
  '9/30/2015 4:12']]


## Extracting Ask HN and Show HN Posts

First, I'll identify posts that begin with either Ask HN or Show HN and separate the data for those two types of posts into different lists. Separating the data makes it easier to analyze in the following steps.


In [6]:
#Initialize empty lists
ask_posts, show_posts, other_posts = [],[],[]

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

#Number of posts in each list
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194



## Calculating the Average Number of Comments for Ask HN and Show HN Posts

Now that I separated ask posts and show posts into different lists, I'll calculate the average number of comments each type of post receives.


In [10]:
#Keep track of counts in AskHN and ShowHN posts
total_ask_comments, total_show_comments = 0,0

for post in ask_posts:
    num_comments = int(post[4])
    total_ask_comments += num_comments
    
print('The average number of comments on AskHN posts is {:.0f}'
      .format(total_ask_comments / len(ask_posts)))

for post in show_posts:
    num_comments = int(post[4])
    total_show_comments += num_comments
    
print('The average number of comments on ShowHN posts is {:.0f}'
      .format(total_show_comments / len(show_posts)))

The average number of comments on AskHN posts is 14
The average number of comments on ShowHN posts is 10




On average, AskHN posts in my sample receive approximately 14 comments, whereas ShowHN posts receive approximately 10. Since ask posts are more likely to receive comments, I'll focus our remaining analysis just on these posts.



## Finding the Amount of AskHN Posts and Comments by Hour Created

Next, I'll determine if I can maximize the amount of comments an AskHN post receives by creating it at a certain time. First, I'll find the amount of AskHN posts created during each hour of day, along with the number of comments those posts received. Then, I'll calculate the average amount of comments AskHN posts created at each hour of the day receive.


In [18]:
result_list = []

for post in ask_posts:
    created_at = post[6]
    num_comments = int(post[4])
    result_list.append([created_at,num_comments])
    
counts_by_hour, comments_by_hour = {}, {}

#Extract the hour from the date, which is the first element of each result row.
for result in result_list:
    date = result[0]
    comment = result[1]
    hour = dt.datetime.strptime(date, '%m/%d/%Y %H:%M').strftime('%H')
    
    if hour in counts_by_hour:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += comment
    else:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = comment
        
counts_by_hour


{'00': 55,
 '01': 60,
 '02': 58,
 '03': 54,
 '04': 47,
 '05': 46,
 '06': 44,
 '07': 34,
 '08': 48,
 '09': 45,
 '10': 59,
 '11': 58,
 '12': 73,
 '13': 85,
 '14': 107,
 '15': 116,
 '16': 108,
 '17': 100,
 '18': 109,
 '19': 110,
 '20': 80,
 '21': 109,
 '22': 71,
 '23': 68}


## Calculating the Average Number of Comments for Ask HN Posts by Hour


In [22]:
avg_by_hour = []

for hour in comments_by_hour:
    avg_by_hour.append([hour, comments_by_hour[hour] / counts_by_hour[hour]])

avg_by_hour

[['11', 11.051724137931034],
 ['22', 6.746478873239437],
 ['01', 11.383333333333333],
 ['21', 16.009174311926607],
 ['18', 13.20183486238532],
 ['08', 10.25],
 ['07', 7.852941176470588],
 ['23', 7.985294117647059],
 ['00', 8.127272727272727],
 ['15', 38.5948275862069],
 ['19', 10.8],
 ['09', 5.5777777777777775],
 ['20', 21.525],
 ['17', 11.46],
 ['14', 13.233644859813085],
 ['13', 14.741176470588234],
 ['04', 7.170212765957447],
 ['02', 23.810344827586206],
 ['05', 10.08695652173913],
 ['06', 9.022727272727273],
 ['03', 7.796296296296297],
 ['16', 16.796296296296298],
 ['10', 13.440677966101696],
 ['12', 9.41095890410959]]

## Sorting and Printing Values from a List of Lists

In [23]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
print(swap_avg_by_hour)

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

sorted_swap

[[11.051724137931034, '11'], [6.746478873239437, '22'], [11.383333333333333, '01'], [16.009174311926607, '21'], [13.20183486238532, '18'], [10.25, '08'], [7.852941176470588, '07'], [7.985294117647059, '23'], [8.127272727272727, '00'], [38.5948275862069, '15'], [10.8, '19'], [5.5777777777777775, '09'], [21.525, '20'], [11.46, '17'], [13.233644859813085, '14'], [14.741176470588234, '13'], [7.170212765957447, '04'], [23.810344827586206, '02'], [10.08695652173913, '05'], [9.022727272727273, '06'], [7.796296296296297, '03'], [16.796296296296298, '16'], [13.440677966101696, '10'], [9.41095890410959, '12']]


[[38.5948275862069, '15'],
 [23.810344827586206, '02'],
 [21.525, '20'],
 [16.796296296296298, '16'],
 [16.009174311926607, '21'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [13.20183486238532, '18'],
 [11.46, '17'],
 [11.383333333333333, '01'],
 [11.051724137931034, '11'],
 [10.8, '19'],
 [10.25, '08'],
 [10.08695652173913, '05'],
 [9.41095890410959, '12'],
 [9.022727272727273, '06'],
 [8.127272727272727, '00'],
 [7.985294117647059, '23'],
 [7.852941176470588, '07'],
 [7.796296296296297, '03'],
 [7.170212765957447, '04'],
 [6.746478873239437, '22'],
 [5.5777777777777775, '09']]

In [24]:
# Sort the values and print the the 5 hours with the highest average comments.

print("Top 5 Hours for 'AskHN' Comments")
for avg, hr in sorted_swap[:5]:
    print(
        "{}: {:.2f} average comments per post".format(
            dt.datetime.strptime(hr, "%H").strftime("%H:%M"),avg
        )
    )



Top 5 Hours for 'AskHN' Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


---

The hour of the day that receives the most comments per post on average is 15:00(3:00 PM), with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts/home), the timezone used in the dataset is Eastern Time in the US. So, you can plan to write your posts by 15:00(3:00 PM) EST.



## Conclusion

In this project, I analyzed AskHN posts and ShowHN posts to determine which type of post and time of the day receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, I'd recommend the post be categorized as an AskHN post and be created between 15:00 and 16:00 (3:00 PM EST - 4:00 PM EST).

However, it should be noted that the data set I analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, AskHN posts received more comments on average and AskHN posts created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) received the most comments on average.
